In [ ]:
import pandas as pd
from modules.data_process import (if_exists, return_data, save_file, add_value_to_column,
                                  return_model, find_index_by_name)
from modules.agents import Agents
model, temperature, opener, escalator = return_model()
agent = Agents(model=model, temperature=temperature, opener=opener, escalator=escalator)

In [ ]:
name = input("Name of the user to mail")
if name and if_exists(str(name)):
    index = find_index_by_name(str(name))
    data = return_data(str(name))
    result = agent.opener.invoke(input={"input": f"{data[0]}"})
    agent.memory.save_context(
        inputs={"input": f"{data[0]}"},
        outputs={"output": result["subject"] + "\n" + result["body"]},
    )
    add_value_to_column(column_name="Subject", value=result["subject"], index=index)
    add_value_to_column(column_name="Body", value=result["body"], index=index)
    save_file()
    agent.get_summary()

In [ ]:
while(True):
    query=input("Chat with ROBO (Enter q to quit)")
    output=""
    if query=="q":
        break
    else:
        results=agent.escalator.invoke({'input':query,
                                       'chat_history':agent.summary})
        if 'output' in results:
            output=results['output']
        if len(output) > 0:
            agent.memory.save_context(inputs={"input": query},
                                  outputs={"output": results['output']})
            add_value_to_column(column_name="lead_status", value="Not Escalated", index=index)
            add_value_to_column(column_name="agent_response", value=output, index=index)
            save_file()
        elif results["status"] == "Escalated":
            agent.memory.save_context(inputs={"input": query},
                                  outputs={"output": results["agent"] + "\n" + results["status"]})
            add_value_to_column(column_name="agent_response", value=None, index=index)
            add_value_to_column(column_name="lead_status", value=results["status"], index=index)
            save_file()

        agent.get_summary()
